In [63]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("../rawdata/rawdata_link.csv")

# Lấy cột product_link dưới dạng list
product_links = df["product_link"].dropna().tolist()
selected_links = [product_links[3]] + product_links[500:600]
# selected_links = product_links[:100]
print("Số lượng link:", len(selected_links))
print(selected_links)

Số lượng link: 101
['https://cellphones.com.vn/laptop-asus-tuf-gaming-a15-fa506ncr-hn047w.html', 'https://cellphones.com.vn/macbook-air-15-inch-m3-16gb-1tb-2024-sac-70w.html', 'https://cellphones.com.vn/laptop-hp-probook-440-g9.html', 'https://cellphones.com.vn/laptop-dell-latitude-e7250.html', 'https://cellphones.com.vn/laptop-lenovo-thinkpad-p51.html', 'https://cellphones.com.vn/laptop-lenovo-ideapad-slim-5-16imh9-83dc001rvn.html', 'https://cellphones.com.vn/apple-mac-mini-m4-pro-12cpu-16gpu-64gb-1tb-2024-ethernet-10-gigabit.html', 'https://cellphones.com.vn/laptop-lenovo-ideapad-slim-5-14akp10-83hx001jvn-ryzen-ai-7.html', 'https://cellphones.com.vn/laptop-lenovo-thinkpad-e16-gen-1-21jn006gvn.html', 'https://cellphones.com.vn/apple-mac-mini-m4-10cpu-10gpu-32gb-256gb-2024.html', 'https://cellphones.com.vn/laptop-gigabyte-g5-kf5-53vn383sh.html', 'https://cellphones.com.vn/laptop-asus-expertbook-p1-p1503cva-i5se16-63ws.html', 'https://cellphones.com.vn/laptop-hp-elitebook-840-g9.html', 

In [64]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import os

# Khởi tạo driver
path = "./chromedriver.exe"
service = Service(path)
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(service=service, options=options)

# Mở trang web
# url = ""
# driver.get(url)
# time.sleep(3)

# data = []

from bs4 import BeautifulSoup

all_data = []

for link in selected_links:
    try:
        driver.get(link)
        time.sleep(1)
        for _ in range(2):
            # Lướt xuống 1 màn hình
            driver.execute_script("window.scrollBy(0, window.innerHeight);")
            time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        data = {}
        data["Title"] = soup.select_one(".box-product-name h1").get_text().strip().replace("\n", "") if soup.select_one(".box-product-name h1") else None
        data["Information"] = soup.select_one(".additional-information").get_text().strip().replace("\n", "") if soup.select_one(".additional-information") else None
        target_div = soup.select_one('div.tpt-box.has-text-centered.is-flex.is-flex-direction-column.is-flex-wrap-wrap.is-justify-content-center.is-align-items-center.active') if soup.select_one('div.tpt-box.has-text-centered.is-flex.is-flex-direction-column.is-flex-wrap-wrap.is-justify-content-center.is-align-items-center.active') else None
        
        # Lấy giá sale
        if target_div:
            sale_price_tag = target_div.select_one('p.tpt---sale-price')
            data['Price sale'] = sale_price_tag.get_text(strip=True) if sale_price_tag else None

            origin_price_tag = soup.select_one('p.tpt---price')
            data['Price origin'] = origin_price_tag.get_text(strip=True) if origin_price_tag else None
        else:
            sale_price_tag = soup.select_one('p.product__price--show')
            data['Price sale'] = sale_price_tag.get_text(strip=True) if sale_price_tag else None

            origin_price_tag = soup.select_one('p.product__price--through')
            data['Price origin'] = origin_price_tag.get_text(strip=True) if origin_price_tag else None
        
        # data['Price sale'] = target_div.select_one('p.tpt---sale-price').get_text() if target_div.select_one('p.tpt---sale-price') else None
        # data['Price origin'] = soup.select_one('p.tpt---price').get_text() if soup.select_one('p.tpt---price') else None
        for item in soup.select("ul.technical-content > li"):
            key = item.select("p")[0].text
            value = item.select("div")[0].text
            data[key] = value

    except Exception as e:
        print(f"Lỗi khi xử lý link: {link}\n{e}")
        continue

    all_data.append(data)

# Ghi dữ liệu ra file CSV
file = "../rawdata/rawdata_cellphones.csv"
df = pd.DataFrame(all_data)
df.to_csv(file, mode='a', encoding="utf-8", header=not os.path.exists(file), index=False)

print(f"Đã lấy {len(all_data)} sản phẩm. Dữ liệu đã lưu vào file.")

# Đóng trình duyệt
driver.quit()

Đã lấy 101 sản phẩm. Dữ liệu đã lưu vào file.
